# Generating CRs :

This file contains a class to generate and alter CRs. It creates an object containing the essential information pertaining to that perticular CR : index, source node, destination node, security level(sl), time-slots required(tk), status('allocated' or 'blocked'), path, and allocated resources.

There are methods to generate CRs, update the status, calculate the k-sp(given a graph g), create priority queue, calculate SRCR/NSP.

There is also a function to plot the number of CRs with a particvular security level vs the number of total CRs. This is to verify the uniformity with respect to different sl of CRs.

In [1]:
## Required Modules
import numpy as np
import random
import math
from collections import Counter
import matplotlib.pyplot as plt

Given the network topology

We have the number of nodes "numNodes" = len(g) #g is a graph


In [3]:
class CR:
    #CRs = []
    sl_values = ["high", "medium", "low"]
    sfw = {"high" : 5, "medium" : 4, "low" : 3}    # Security factor weight
    
    def __init__(self, index, s, d, sl, tk, status):     # Constructor : Gets called as soon as an object is declared
        self.index = index
        
        self.s = s
        self.d = d
        
        self.sl = sl    # SL can be hign, mediuum, low
        self.tk = tk
        self.status = status    # Status can be Allocatted, blocked

        self.path = None
        self.allocated_resources = [None, None]    # [QSC, ts]
        cr.text = ''
        

    def update_status(self, status, allocated_resources = [None, None], path = None) :    # A method to update the status

        self.status = status

        self.path = path
        self.allocated_resources = allocated_resources

            
    def k_sp(self, g, k = 1):
        try:
            # Find the K-shortest paths
            k_shortest_paths = list(nx.shortest_simple_paths(g, source = self.s, target = self.d, weight='weight'))

            # Retrieve path costs
            path_costs = [sum(g[s][d]['weight'] for s, d in zip(path, path[1:])) for path in k_shortest_paths]
        
           # Return both paths and corresponding path costs
            return list(zip(k_shortest_paths, path_costs))[:k]
            
        except nx.NetworkXNoPath:    # i.e. the node is not connected to the given network
            # Handle the case where no path exists
            return []
            
    
    def shortest_path(self, g): # shortest path in terms of hop counts
        return nx.shortest_path(g, source = self.s, target = self.d)

    
    def display_info(self):
        print(f"CR: Index : {self.index}, Source Node ={self.s}, Destination Node={self.d}, Security Level={self.sl}, status={self.status}, allocated resources = {self.allocated_resources}, Path : {self.path}")
 
  ###################################################################################################################################################
    
    @classmethod
    def generate_crs(cls, numCR = 1):    
        cls.CRs = []
    
        # Creating a list of uniformly sampled SLs
        n = math.floor(numCR/3)
        uniform_sample = n*cls.sl_values

        for i in range(numCR - 3*n):    # the range would be either 1 or 2. i can either be 0 or (0, 1)
             uniform_sample.append(random.choice(cls.sl_values))

        # Initializing individual CRs
        for i in range(numCR):
            # Generating (source, destination) pair
            s = random.randint(1, numNodes)
            d = random.randint(1, numNodes)   

            while s == d :    # Checking if the source and destination nodes are same
                d = random.randint(1, numNodes)  

            # Assigning sl            
            sl = random.choice(uniform_sample)    # Randomly selecting from a uniform sample
            uniform_sample.remove(sl)    # Removing the first(any) equivalent sl, to maintain the uniformity

            # currently only dealing with tk = 1.
            tk = 1
            
            # Giving initial status
            status = "initialized"

            cr = cls(i, s, d, sl, tk, status)
            cls.CRs.append(cr)
        ##
        return cls.CRs

    @classmethod
    def create_priority_queue(cls, queues = 'PQ'):
        CR_1 = []
        CR_0 = []
        CR_minus1 = []

        for cr in cls.CRs :
            if cr.sl == "high":
                CR_1.append(cr)
            elif cr.sl == "medium":
                CR_0.append(cr)
            else:
                CR_minus1.append(cr)

        PQ = CR_1 + CR_0 + CR_minus1    # Priority queue

        if queues == 'all':
            return PQ, CR_1, CR_0, CR_minus1
        else:
            return PQ
            

    @classmethod
    def SRCR(cls, CRs):    # Returns the SRCR

        num_allocated_crs = sum(1 for cr in CRs if cr.status == "allocated" )
        
        SRCR = num_allocated_crs/len(CRs)
        print(f"The Success rate of connection requests(SRCR) is : {SRCR}")
        
        return SRCR
        

    @classmethod
    def NSP(cls, CRs, channel = 'total'):    # Returns the total NSP, if no 2nd argument given

        nsp = [0, 0, 0, 0]
        for i, priority in enumerate(cls.sl_values) :
            nsp[i+1] = sum(cls.sfw[priority] for cr in CRs if cr.status == "allocated" and cr.sl == priority)
            nsp[0] += nsp[i+1]

        print(f"The NSP of the network is : {nsp[0]}")

        if channel == 'all':
            return nsp
        else :
            return nsp[0]

    
    @classmethod
    def display_all(cls, CRs):
        for cr in CRs:
            cr.display_info()


The above class CR has a constructor to initialize the values to its object. It has a class method "generate_crs" which takes the numCR parameter to generate that many CRs. the (s, d) pair is selected randomly. The sl is derived from a uniform list; and is allocated randomly to individual CR. 

In [4]:
def plot_crs(*CRs) :

    high = []
    mid = []
    low = []
    numNodes = 14
    for X in range(25, 850, 25):
        CRs = CR.generate_crs(X) 
        counts = dict(Counter([CR.sl for CR in CRs]))
        high.append(counts["high"])
        mid.append(counts["medium"])
        low.append(counts["low"])

    plt.plot(range(25, 850, 25), high, label = "high")
    plt.plot(range(25, 850, 25), mid, label = "mid")
    plt.plot(range(25, 850, 25), low, label = "low")
    
    plt.scatter(range(25, 850, 25), high, label = "high")
    plt.scatter(range(25, 850, 25), mid, label = "mid")
    plt.scatter(range(25, 850, 25), low, label = "low")
    
    plt.legend(loc = 'best')
    plt.title("Distribution of SL levels for various number of CRs")
    plt.xlabel("Num of CRs")
    plt.ylabel("Counts of SL")